# Regression and DNN Model 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pydot
import os


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

print(tf.__version__)

# Get Dataset

In [ ]:
dir_p = r'C:\Users\Honeyz\Desktop\Aessa\THE_SIS\PyroDataProcessed.csv'
raw_dataset = pd.read_csv(dir_p, skipinitialspace=True)

dataset = raw_dataset.copy()
dataset = dataset.dropna()
dataset

## Check Null values

In [ ]:
#The dataset contains a few unknown values.
dataset.isna().sum()

In [ ]:
#Drop those rows to keep this initial tutorial simple.
#dataset = dataset.dropna()

# Splitting Data to training and testing data

In [ ]:
#train_dataset = dataset.sample(frac=0.8, random_state=0)

#test_dataset = dataset.drop(train_dataset.index)



In [ ]:
#dataset.describe().transpose()

In [ ]:
#In the table of statistics it's easy to see how different the ranges of each feature are.
dataset.describe().transpose()[['mean', 'std']]

In [ ]:
loss_per_fold = []

# Keras Functional API Model


In [ ]:


def build_and_compile_model_0(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(3, activation='linear')
  ])

    model.compile(loss=['mean_absolute_error'], optimizer=tf.keras.optimizers.Adam(0.001),metrics=[keras.metrics.MeanAbsoluteError()]
    )
    return model

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=247, shuffle=False)

# K-fold Cross Validation model evaluation
fold_no = 1

for train, test in kfold.split(dataset):
    
    train_features = dataset.reindex(train)
    test_features = dataset.reindex(test)


    train_labels = train_features.iloc[:, 16:].values
    train_features = train_features.iloc[:, :16].values
    
    test_labels = test_features.iloc[:, 16:].values
    test_features = test_features.iloc[:, :16].values


    normalizer = preprocessing.Normalization()
    normalizer.adapt(np.array(train_features))

    dnn_model = build_and_compile_model_0(normalizer)
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    history = dnn_model.fit(train_features, y= (train_labels), batch_size=64, epochs=150, validation_split=0.1)

    # Generate generalization metrics
    
    scores = dnn_model.evaluate(test_features, (test_labels), verbose=0)
    print(f'Score for fold {fold_no}: {dnn_model.metrics_names[0]} of {scores[0]}; {dnn_model.metrics_names[1]} of {scores[1]*100}%')
    
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
dnn_model.summary()
keras.utils.plot_model(dnn_model, "multi_input_and_output_model.png", show_shapes=True)


# Model Evaluation

In [ ]:

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 40])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
print(plot_loss(history))

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 40) 
plt.show()

In [ ]:
dnn_model.evaluate(test_features, y=(test_labels_0,test_labels_1), verbose=0)

In [ ]:
test_predictions = dnn_model.predict(test_features)

a = plt.axes(aspect='equal')
plt.scatter(test_labels_0, test_predictions[0])
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0,80]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


In [ ]:
np.random.__RandomState_ctor